# 加载库函数及导入文件

In [44]:
%run corelibs.py
# import pathlib
# import pandas as pd
OUTPUT_DIR_PATH = pathlib.Path(r"E:\张楠\测试\output")
print(output_dir_path)

E:\张楠\测试\output


# 测试

## 提取excel表头和md5值

In [12]:
%%time
from hashlib import md5
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_16_1_watermark.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\建行第二次查询7.13\1080861701689234045070165_1_1田桂芬.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
print(a := read_header(acc_file))
md5(a.encode()).hexdigest()

('序号', '客户名称', '账号', '币种', '钞汇', '册号', '笔号', '交易日期', '交易时间', '活期存款账户明细号', '交易卡号', '摘要', '借贷方向', '交易金额', '账户余额', '柜员号', '交易机构号', '交易机构名称', '对方账号', '对方户名', '对方行名', '交易流水号', '交易渠道', '自助设备编号', '扩充备注', 'IP地址', 'MAC地址', '第三方订单号', '商户号', '商户名称')
Wall time: 951 ms


D:\soft\WPy64-3890\python-3.8.9.amd64\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


'8fa9c0776b857793f91335192398aeba'

## 测试yaml配置文件语法

In [101]:
import yaml
with open('config.yaml', 'r', encoding='utf-8') as f:
    CONF_DATA = yaml.safe_load(f)

In [ ]:
CONF_DATA

## 测试配置转换内部操作表示功能

In [ ]:
%run corelibs.py
conf_obj = creat_conf_obj(CONF_DATA['建设银行网点']['流水'])

conf_obj

##  测试单个账户文件读取及储存

In [25]:
%run corelibs.py
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_15_1时燕.xlsx")
df = process_account_file_general(acc_file, OUTPUT_DIR_PATH, '建设银行网点', '账户')

In [ ]:
import dtale

In [108]:
dtale.show(df, open_browser=True)

## 测试单个流水文件的读取与储存

In [39]:
%run corelibs.py
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_16_1_watermark.xlsx")
df = process_statment_file_general(acc_file, OUTPUT_DIR_PATH, '建设银行网点', '流水', '1111111-11')

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
import dtale

In [16]:
dtale.show(df, open_browser=True)

# 执行

## 分析建设银行网点数据目录
（如：开发区金融街建行查询数据）

In [43]:
%run corelibs.py
from collections import Counter
from hashlib import md5
from tqdm.notebook import tqdm as tqnb

base_dir = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人")

def _process_files_general(files_list: list, output_dir: pathlib.Path, doc_No: str=None) -> int:
        for _file in tqnb(files_list):
            _header = read_header(_file).encode() # 读取每个文件的表头
            _conf_name = HEADER_HASH.get(md5(_header).hexdigest()) # 根据表头md5值找到相应的配置
            if _conf_name is None:
                print(f'警告：{_file.name}未找到对应配置，跳过')
                continue
            elif _conf_name[1] == '账户':
                print(f'　　　{_file.name}……', end='')
                process_account_file_general(_file, output_dir, _conf_name[0], '账户')
                print(f'{"".join(_conf_name)}done')
            elif _conf_name[1] == '流水':
                print(f'　　　{_file.name}……', end='')
                process_statment_file_general(_file, output_dir, _conf_name[0], '流水', doc_No)
                print(f'{"".join(_conf_name)}done')
            else:
                 raise Exception("header_hash配置有误") 
        return len(files_list)

def process_dir_ccb_branch(dir_path: pathlib.Path, output_dir: pathlib.Path, doc_No: str=None) -> None:
    """"""
    # 分析是否存在因为超过9999条而分文件保存的流水文件
    if dir_path.is_dir():
        _file_names = list(dir_path.glob('[!~]*.xlsx')) # 找到目录中所有的excel文件（不含子目录）
         # 检查是否存在超过9999条的数据（文件名第二字段的数字一样）
        _d, _c = Counter(map(lambda x: x.name.split('_')[1], _file_names)).most_common(1)[0]
        if _c > 1: 
            raise Exception(f'目录中包含分开保存的同账户流水文件，需要手动合并，文件名第二字段为{_d}')
        print(f'结果：共处理{_process_files_general(_file_names, output_dir, doc_No)}个文件')
    else:
        raise Exception("传入的路径不是目录，请传入目录路径，或使用单文件分析") 

process_dir_ccb_branch(base_dir, OUTPUT_DIR_PATH, 'test_ccb_dir')

  0%|          | 0/21 [00:00<?, ?it/s]

　　　1080861461690963794022777_10_1_watermark.xlsx……

  0%|          | 0/1 [00:00<?, ?it/s]

建设银行网点流水done
　　　1080861461690963794022777_11_1权威.xlsx……建设银行网点账户done
　　　1080861461690963794022777_12_1_watermark.xlsx……

Exception: 验证未通过，需清洗数据列：交易时间

# 杂项

In [48]:
pd.read_excel

<function pandas.io.excel._base.read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, mangle_dupe_cols=True, storage_options: Union[Dict[str, Any], NoneType] = None)>

In [6]:
hash(str(['所属银行','证件号码','开户名称','客户账号','卡号','余额','开户日期','开户机构名称','是否开通网易','U盾号','短信捆绑电话','网银电话','地址','邮编','销户日期','销户机构号','销户机构名称','账户状态','我行账号（卡号）','我方银行名称','我方中文名称','身份证号','交易日期','对方账号（卡号）','对方中文名称','对方银行名称','对方银行机构代码','流水号','借贷标志','交易金额','币种','余额','网银IP地址','交易代码（交易渠道）','代码描述','用途（摘要）','受理机构代码','受理机构名称','交易是否成功']))

3955050809677091182

In [124]:
type(CONF_DATA['天津银行']['流水']['时间']['交易日期'])

TypeError: list indices must be integers or slices, not str

In [ ]:
CONF_DATA['天津银行']['流水']

In [27]:
print(CONF_DATA.get('哈哈'))

In [37]:
from itertools import islice
a = [3,4,5,6,7,8]
b = islice(a,4,5)
next(b)

In [34]:
 list(base_dir.glob('[!~]*.xlsx'))

[WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_10_1_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_11_1权威.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_12_1_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_13_1苏文.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_14_1_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_15_1时燕.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_161_2_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_162_3_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_163_4_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/1080861461690963794022777_16_1_watermark.xlsx'),
 WindowsPath('E:/张楠/2023赵益华927/银行/建行/第一批 赵益华等9人/108